In [37]:
import numpy as np
import keras
from keras.layers import Dense, Flatten, Dropout, Lambda, Input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications.resnet50 import ResNet50
from keras.applications import resnet50
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
import warnings
warnings.filterwarnings('ignore')

In [9]:
import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
KTF.set_session(tf.Session(config=tf.ConfigProto(device_count={'gpu':0})))

In [10]:
import os
trainCount = 0
validCount = 0


trainFolder = 'train_data/train'
for sub_folder in os.listdir(trainFolder):
    path, dirs, files = next(os.walk(os.path.join(trainFolder,sub_folder)))
    trainCount += len(files)


validFolder = 'train_data/test'
for sub_folder in os.listdir(validFolder):
    path, dirs, files = next(os.walk(os.path.join(validFolder,sub_folder)))
    validCount += len(files)
    
print("train count=", trainCount)
print('valid count=', validCount)

train count= 381
valid count= 164


In [38]:
input_img = Input(shape = (48, 48, 3))
x = Lambda(resnet50.preprocess_input)(input_img)
ResNet50_model = ResNet50(input_tensor = x, weights = 'imagenet', include_top = False)
ResNet50_model.trainable = False;
x = Flatten()( ResNet50_model.output)
predictions = Dense(2, activation="softmax")(x)
finalModel = Model(input = ResNet50_model.input, output = predictions)



In [12]:
finalModel.compile(loss = 'binary_crossentropy',
                          optimizer = 'Adam',
                          metrics = ['accuracy'])

In [13]:
height, width = 48, 48
batchSize = 32
train_datagen = ImageDataGenerator(
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.1,
width_shift_range = 0.1,
height_shift_range=0.1,
rotation_range=5)

valid_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
trainFolder,
target_size = (height, width),
batch_size = batchSize,
class_mode = "categorical")

validation_generator = valid_datagen.flow_from_directory(
validFolder,
target_size = (height, width),
batch_size = batchSize,
class_mode = "categorical")

checkpointer = ModelCheckpoint("parking.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early_stopping = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')

Found 381 images belonging to 2 classes.
Found 164 images belonging to 2 classes.


In [14]:
finalModel.fit_generator(
    train_generator,
    epochs = 20,
    steps_per_epoch = trainCount // batchSize,
    validation_data = validation_generator,
    validation_steps = validCount // batchSize,
    verbose = 1,
    callbacks = [checkpointer, early_stopping])

Epoch 1/20
11/11 [==============================] - 37s 3s/step - loss: 0.2117 - acc: 0.9261 - val_loss: 2.4890 - val_acc: 0.8313

Epoch 00001: val_acc improved from -inf to 0.83125, saving model to parking.h5
Epoch 2/20
11/11 [==============================] - 2s 203ms/step - loss: 0.1390 - acc: 0.9656 - val_loss: 1.1832 - val_acc: 0.9242

Epoch 00002: val_acc improved from 0.83125 to 0.92424, saving model to parking.h5
Epoch 3/20
11/11 [==============================] - 2s 146ms/step - loss: 0.1503 - acc: 0.9741 - val_loss: 0.7287 - val_acc: 0.9545

Epoch 00003: val_acc improved from 0.92424 to 0.95455, saving model to parking.h5
Epoch 4/20
11/11 [==============================] - 2s 141ms/step - loss: 0.0335 - acc: 0.9915 - val_loss: 0.6453 - val_acc: 0.9545

Epoch 00004: val_acc did not improve from 0.95455
Epoch 5/20
11/11 [==============================] - 1s 136ms/step - loss: 0.1029 - acc: 0.9886 - val_loss: 1.2714 - val_acc: 0.9091

Epoch 00005: val_acc did not improve from 0.

In [28]:
from keras.preprocessing import image
def path_to_tensor(imgPath):
    img = image.load_img(imgPath, target_size=(48, 48))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return img

In [23]:
print(train_generator.class_indices)

{'empty': 0, 'occupied': 1}


In [36]:
pred_result = finalModel.predict(path_to_tensor('./cnn_data/spot359.jpg'))
pred_result = pred_result.clip(min=0.005, max=0.995)
index = np.argmax(pred_result[0])
print(index)
print(pred_result[0][1])

1
0.995


1
0.99999905
